In [1]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import regularizers
import os
import pandas as pd
import time
from tensorflow.keras.models import load_model
from sklearn.decomposition import PCA

In [15]:
mylist= os.listdir('C:/Users/user/Desktop/sentimentaudio/female')

In [18]:
mylist

['trainJEJ00_1_001.mp3_segment_1.wav',
 'trainJEJ00_1_001.mp3_segment_10.wav',
 'trainJEJ00_1_001.mp3_segment_2.wav',
 'trainJEJ00_1_001.mp3_segment_3.wav',
 'trainJEJ00_1_001.mp3_segment_4.wav',
 'trainJEJ00_1_001.mp3_segment_5.wav',
 'trainJEJ00_1_001.mp3_segment_6.wav',
 'trainJEJ00_1_001.mp3_segment_7.wav',
 'trainJEJ00_1_001.mp3_segment_8.wav',
 'trainJEJ00_1_001.mp3_segment_9.wav',
 'trainJEJ00_2_002.mp3_segment_1.wav',
 'trainJEJ00_2_002.mp3_segment_10.wav',
 'trainJEJ00_2_002.mp3_segment_2.wav',
 'trainJEJ00_2_002.mp3_segment_3.wav',
 'trainJEJ00_2_002.mp3_segment_4.wav',
 'trainJEJ00_2_002.mp3_segment_5.wav',
 'trainJEJ00_2_002.mp3_segment_6.wav',
 'trainJEJ00_2_002.mp3_segment_7.wav',
 'trainJEJ00_2_002.mp3_segment_8.wav',
 'trainJEJ00_2_002.mp3_segment_9.wav',
 'trainJEJ00_3_003.mp3_segment_1.wav',
 'trainJEJ00_3_003.mp3_segment_10.wav',
 'trainJEJ00_3_003.mp3_segment_2.wav',
 'trainJEJ00_3_003.mp3_segment_3.wav',
 'trainJEJ00_3_003.mp3_segment_4.wav',
 'trainJEJ00_3_003.mp3

In [11]:
from pydub import AudioSegment
import os

# Specify the input and output folders
input_folder = "C:/Users/user/Desktop/sentimentaudio/"
output_folder = "C:/Users/user/Desktop/sentimentaudio/"

# Make sure the output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Iterate through all files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".mp3"):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)
        print(input_path)
        # Load the audio file
        #audio = AudioSegment.from_file(input_path,'mp3')
        audio = AudioSegment.from_mp3(input_path)

        # Set the target duration
        target_duration = 30.5
        target_length = int(target_duration * 1000)  # Convert seconds to milliseconds

        # Adjust the duration using a conditional expression
        adjusted_audio = audio[:target_length] if len(audio) > target_length else audio

        # Export the adjusted audio to the output folder
        adjusted_audio.export(output_path, format="mp3")

C:/Users/user/Desktop/sentimentaudio/trainJEJ00_1_001.mp3
C:/Users/user/Desktop/sentimentaudio/trainJEJ00_2_002.mp3
C:/Users/user/Desktop/sentimentaudio/trainJEJ00_3_003.mp3
C:/Users/user/Desktop/sentimentaudio/trainJEJ00_4_004.mp3
C:/Users/user/Desktop/sentimentaudio/trainJEJ00_5_005.mp3
C:/Users/user/Desktop/sentimentaudio/trainJEJ00_6_006.mp3
C:/Users/user/Desktop/sentimentaudio/trainJHW01_1_007.mp3
C:/Users/user/Desktop/sentimentaudio/trainJHW01_2_008.mp3
C:/Users/user/Desktop/sentimentaudio/trainJHW01_3_009.mp3
C:/Users/user/Desktop/sentimentaudio/trainJHW01_4_010.mp3
C:/Users/user/Desktop/sentimentaudio/trainJHW01_5_011.mp3
C:/Users/user/Desktop/sentimentaudio/trainJHW01_6_012.mp3
C:/Users/user/Desktop/sentimentaudio/trainLCY00_1_013.mp3
C:/Users/user/Desktop/sentimentaudio/trainLCY00_2_014.mp3
C:/Users/user/Desktop/sentimentaudio/trainLCY00_3_015.mp3
C:/Users/user/Desktop/sentimentaudio/trainLCY00_4_016.mp3
C:/Users/user/Desktop/sentimentaudio/trainLCY00_5_017.mp3
C:/Users/user/

In [34]:
input_path

'C:/Users/user/Desktop/sentimentaudio/trainSEJ00_6_054.mp3'

In [ ]:
from pydub import AudioSegment
import os

def cut_and_save_both_halves(input_folder, output_folder, segment_duration=3000):
    # Get a list of audio files in the input folder
    audio_files = [f for f in os.listdir(input_folder) if f.endswith('.mp3')]

    os.makedirs(output_folder, exist_ok=True)

    for audio_file in audio_files:
        # Construct the full path to the input audio file
        input_path = os.path.join(input_folder, audio_file)

        # Load the audio file
        sound = AudioSegment.from_file(input_path)

        # Calculate the number of segments
        num_segments = len(sound) // segment_duration

        for i in range(num_segments):
            start_time = i * segment_duration
            end_time = (i + 1) * segment_duration

            # Extract the segment
            segment = sound[start_time:end_time]

            # Construct the full path to the output audio file for the first half
            first_half_output_path = os.path.join(output_folder, f"{audio_file}_segment_{i + 1}_first_half.mp3")
            
            # Construct the full path to the output audio file for the second half
            second_half_output_path = os.path.join(output_folder, f"{audio_file}_segment_{i + 1}_second_half.mp3")

            # Export the first half as an mp3 file
            segment.export(first_half_output_path, format="mp3")

            # Export the second half as an mp3 file
            second_half = sound[end_time:]
            second_half.export(second_half_output_path, format="mp3")

# Example usage:
input_folder_path = "C:/Users/user/Desktop/sentimentaudio/"
output_folder_path = "C:/Users/user/Desktop/sentimentaudio/segmented_audio"
cut_and_save_both_halves(input_folder_path, output_folder_path, segment_duration=10000)

In [19]:
## 감정별 성별 분류
feeling_list=[]
for item in mylist:
    if item[8:10] == "00" and item[11] == "1":
        feeling_list.append("female_angry")
    elif item[8:10] == "01"and item[11] == "1":
        feeling_list.append("male_angry")

    elif item[8:10] == "00" and item[11] == "2":
        feeling_list.append("female_confident")
    elif item[8:10] == "01" and item[11] == "2":
        feeling_list.append("male_confident")
        
    elif item[8:10] == "00" and item[11] == "3":
        feeling_list.append("female_happy")
    elif item[8:10] == "01" and item[11] == "3":
        feeling_list.append("male_happy")
        
    elif item[8:10] == "00" and item[11] == "4":
        feeling_list.append("female_nervous")
    elif item[8:10] == "01" and item[11] == "4":
        feeling_list.append("male_nervous")
        
    elif item[8:10] == "00" and item[11] == "5":
        feeling_list.append("female_neutral")
    elif item[8:10] == "01" and item[11] == "5":
        feeling_list.append("male_neutral")
        
    elif item[8:10] == "00" and item[11] == "6":
        feeling_list.append("female_sad")
    elif item[8:10] == "01" and item[11] == "6":
        feeling_list.append("male_sad")

In [21]:
feeling_list

238

In [22]:
labels = pd.DataFrame(feeling_list)

In [23]:
labels.sample(10)

,0
45,female_neutral
117,female_sad
231,female_sad
91,female_nervous
3,female_angry
34,female_nervous
38,female_nervous
62,female_angry
84,female_happy
230,female_sad


In [27]:
## Librosa 라이브러리를 이용해 Mel-Frequency Cepstral Coefficients(MFCCs) 추출

start = time.time()
df = pd.DataFrame(columns=['feature'])
bookmark=0
for index,y in enumerate(mylist):
    if mylist[index][11] == '1':
        X, sample_rate = librosa.load('C:/Users/user/Desktop/sentimentaudio/female/'+y, res_type='kaiser_fast',duration=3,sr=22050*2,offset=0.5)
        sample_rate = np.array(sample_rate)
        mfccs = np.mean(librosa.feature.mfcc(y=X, 
                                            sr=sample_rate, 
                                            n_mfcc=13),
                                            axis=0)
        feature = mfccs
        #[float(i) for i in feature]
        #feature1=feature[:135]
        df.loc[bookmark] = [feature]
        bookmark=bookmark+1
        
    elif mylist[index][11] == '2':
        X, sample_rate = librosa.load('C:/Users/user/Desktop/sentimentaudio/female/'+y, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
        sample_rate = np.array(sample_rate)
        mfccs = np.mean(librosa.feature.mfcc(y=X, 
                                            sr=sample_rate, 
                                            n_mfcc=13),
                                            axis=0)
        feature = mfccs
        #[float(i) for i in feature]
        #feature1=feature[:135]
        df.loc[bookmark] = [feature]
        bookmark=bookmark+1
       
    elif mylist[index][11] == '3':
        X, sample_rate = librosa.load('C:/Users/user/Desktop/sentimentaudio/female/'+y, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
        sample_rate = np.array(sample_rate)
        mfccs = np.mean(librosa.feature.mfcc(y=X, 
                                            sr=sample_rate, 
                                            n_mfcc=13),
                                            axis=0)
        feature = mfccs
        #[float(i) for i in feature]
        #feature1=feature[:135]
        df.loc[bookmark] = [feature]
        bookmark=bookmark+1
        
    elif mylist[index][11] == "4":
        X, sample_rate = librosa.load('C:/Users/user/Desktop/sentimentaudio/female/'+y, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
        sample_rate = np.array(sample_rate)
        mfccs = np.mean(librosa.feature.mfcc(y=X, 
                                            sr=sample_rate, 
                                            n_mfcc=13),
                                            axis=0)
        feature = mfccs
        #[float(i) for i in feature]
        #feature1=feature[:135]
        df.loc[bookmark] = [feature]
        bookmark=bookmark+1
       
    elif mylist[index][11] == "5":
        X, sample_rate = librosa.load('C:/Users/user/Desktop/sentimentaudio/female/'+y, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
        sample_rate = np.array(sample_rate)
        mfccs = np.mean(librosa.feature.mfcc(y=X, 
                                            sr=sample_rate, 
                                            n_mfcc=13),
                                            axis=0)
        feature = mfccs
        #[float(i) for i in feature]
        #feature1=feature[:135]
        df.loc[bookmark] = [feature]
        bookmark=bookmark+1
        
    elif mylist[index][11] == "6":
        X, sample_rate = librosa.load('C:/Users/user/Desktop/sentimentaudio/female/'+y, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
        sample_rate = np.array(sample_rate)
        mfccs = np.mean(librosa.feature.mfcc(y=X, 
                                            sr=sample_rate, 
                                            n_mfcc=13),
                                            axis=0)
        feature = mfccs
        #[float(i) for i in feature]
        #feature1=feature[:135]
        df.loc[bookmark] = [feature]
        bookmark=bookmark+1

print("time:", time.time() - start)

C:\Users\user\AppData\Local\Temp\ipykernel_17204\1373056004.py:8: UserWarning: PySoundFile failed. Trying audioread instead.
  X, sample_rate = librosa.load('C:/Users/user/Desktop/sentimentaudio/female/'+y, res_type='kaiser_fast',duration=3,sr=22050*2,offset=0.5)
C:\Users\user\anaconda3\Lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
C:\Users\user\anaconda3\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
C:\Users\user\AppData\Local\Temp\ipykernel_17204\1373056004.py:8: UserWarning: PySoundFile failed. Trying audioread instead.
  X, sample_rate = librosa.load('C:/Users/user/Desktop/sentimentaudio/female/'+y, res_type='kaiser_fast',duration=3,sr=22050*2,offset=0.5)
C:\Users\user\anaconda3\Lib\site-packages\libr

C:\Users\user\AppData\Local\Temp\ipykernel_17204\1373056004.py:73: UserWarning: PySoundFile failed. Trying audioread instead.
  X, sample_rate = librosa.load('C:/Users/user/Desktop/sentimentaudio/female/'+y, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
C:\Users\user\anaconda3\Lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
C:\Users\user\anaconda3\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
C:\Users\user\AppData\Local\Temp\ipykernel_17204\1373056004.py:73: UserWarning: PySoundFile failed. Trying audioread instead.
  X, sample_rate = librosa.load('C:/Users/user/Desktop/sentimentaudio/female/'+y, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
C:\Users\user\anaconda3\Lib\site-package

C:\Users\user\AppData\Local\Temp\ipykernel_17204\1373056004.py:60: UserWarning: PySoundFile failed. Trying audioread instead.
  X, sample_rate = librosa.load('C:/Users/user/Desktop/sentimentaudio/female/'+y, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
C:\Users\user\anaconda3\Lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
C:\Users\user\anaconda3\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
C:\Users\user\AppData\Local\Temp\ipykernel_17204\1373056004.py:60: UserWarning: PySoundFile failed. Trying audioread instead.
  X, sample_rate = librosa.load('C:/Users/user/Desktop/sentimentaudio/female/'+y, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
C:\Users\user\anaconda3\Lib\site-package

C:\Users\user\AppData\Local\Temp\ipykernel_17204\1373056004.py:47: UserWarning: PySoundFile failed. Trying audioread instead.
  X, sample_rate = librosa.load('C:/Users/user/Desktop/sentimentaudio/female/'+y, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
C:\Users\user\anaconda3\Lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
C:\Users\user\anaconda3\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
C:\Users\user\AppData\Local\Temp\ipykernel_17204\1373056004.py:47: UserWarning: PySoundFile failed. Trying audioread instead.
  X, sample_rate = librosa.load('C:/Users/user/Desktop/sentimentaudio/female/'+y, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
C:\Users\user\anaconda3\Lib\site-package

time: 0.9992771148681641


C:\Users\user\AppData\Local\Temp\ipykernel_17204\1373056004.py:34: UserWarning: PySoundFile failed. Trying audioread instead.
  X, sample_rate = librosa.load('C:/Users/user/Desktop/sentimentaudio/female/'+y, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
C:\Users\user\anaconda3\Lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
C:\Users\user\anaconda3\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
C:\Users\user\AppData\Local\Temp\ipykernel_17204\1373056004.py:34: UserWarning: PySoundFile failed. Trying audioread instead.
  X, sample_rate = librosa.load('C:/Users/user/Desktop/sentimentaudio/female/'+y, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
C:\Users\user\anaconda3\Lib\site-package

In [5]:
labels = pd.DataFrame(feeling_list)

In [8]:
df_list = pd.DataFrame(df['feature'].values.tolist())

In [9]:
new_df = pd.concat([df_list,labels], axis=1)

In [10]:
rnew_df = new_df.rename(index=str, columns={"0": "label"})

In [11]:
rnew_df.to_csv("Train_neutral_Vector.csv", index=False)